In [2]:
from models.RandomWalk import RandomWalk
from data.data_processing import data_processing
from data.data_transforms import data_transform_std
from data.data_splitting import make_input_output_sequences,train_test_split,shift_sequence
import pandas as pd
import torch
import numpy as np
import torch.utils.data as data_utils
from utils.metrics import MAE,SMAPE
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

horizons = 4
test_ratio = 0.5
file_name = 'datasets/covid_weekly.csv'
data = pd.read_csv(file_name)
target = 'new_deaths'
features = data.columns
m = ["RandomWalk"]
batch_size =32
seq_len = 2
in_dim = len(features)-1

In [3]:
data, observed = data_processing(file_name, features,target)
scalers, df = data_transform_std(data, test_ratio)
x, y = make_input_output_sequences(data.values, seq_len, horizons, True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_ratio)
model =RandomWalk(seq_len,horizons,in_dim)

In [6]:
for w in range(x_test.shape[0]):
    x_train_tensor = torch.from_numpy(np.array(x_train[:,:,1:], dtype=np.float32)).float().to(device)
    y_train_tensor = torch.from_numpy(np.array(y_train[:,:,1:], dtype=np.float32)).float().to(device)
    x_test_tensor = torch.from_numpy(np.array(x_test[:,:,1:], dtype=np.float32)).float().to(device)
    y_test_tensor= torch.from_numpy(np.array(y_test[:,:,1:], dtype=np.float32)).float().to(device)
    train = data_utils.TensorDataset(x_train_tensor, y_train_tensor)
    train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=False)
    
    for epoch in range(20):
            for x_t, y_t in train_loader:
                y_pred = model(x_t,_,_,_)
y_test_pred = model(x_test_tensor,_,_,_)
print(type(y_test_pred))
norm_pred=scalers['scaler_new_deaths'].inverse_transform(y_test_pred[:,:,0].cpu().detach().numpy())
norm_true=scalers['scaler_new_deaths'].inverse_transform(y_test_tensor[:,:,0].cpu().detach().numpy())

smape = [SMAPE(norm_true[:,h],norm_pred[:,h]) for h in range(4)]
print(smape,np.mean(smape))

normMae= [MAE(norm_true[:,h],norm_pred[:,h]) for h in range(4)]
print(normMae, np.mean(normMae))

preds = y_test_pred[:,:,0].cpu().detach().numpy()
trues=y_test_tensor[:,:,0].cpu().detach().numpy()
Mae = [MAE(trues[:,h], preds[:,h]) for h in range(4)]
print(Mae, np.mean(Mae))

<class 'torch.Tensor'>
[15.13975887763791, 19.46467771762755, 24.743415088188357, 30.11031267119617] 22.364541088662495
[807.439, 1160.3903, 1500.1342, 1821.8292] 1322.4481
[0.13769867, 0.1978901, 0.2558292, 0.31069028] 0.22552705


In [4]:
#ROLLING WINDOW for Random WALK
ListPred = []
ListTrue = []
normListPred = []
normListTrue = []
for w in range(x_test.shape[0]):
    x_train_tensor = torch.from_numpy(np.array(x_train[:,:,1:], dtype=np.float32)).float().to(device)
    y_train_tensor = torch.from_numpy(np.array(y_train[:,:,1:], dtype=np.float32)).float().to(device)
    x_test_tensor = torch.from_numpy(np.array(x_test[:,:,1:], dtype=np.float32)).float().to(device)
    y_test_tensor= torch.from_numpy(np.array(y_test[:,:,1:], dtype=np.float32)).float().to(device)
#     print(y_test_tensor)
    train = data_utils.TensorDataset(x_train_tensor, y_train_tensor)
    train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=False)

    for epoch in range(20):
        for x_t, y_t in train_loader:
            y_pred = model(x_t,_,_,_)
        y_test_pred = model(x_test_tensor,_,_,_)
        ListTrue.append(scalers['scaler_new_deaths'].inverse_transform(y_test_tensor[:, :, 0].cpu().detach().numpy())[0,:])
        ListPred.append(scalers['scaler_new_deaths'].inverse_transform(y_test_pred[:,:,0].cpu().detach().numpy())[0,:])
        normListTrue.append((y_test_tensor[:, :, 0].cpu().detach().numpy())[0,:])
        normListPred.append((y_test_pred[:,:,0].cpu().detach().numpy())[0,:])
    x_train, x_test, y_train, y_test = shift_sequence(x_train, y_train, x_test, y_test, 1, True)
ListTrue = np.array(ListTrue)
ListPred = np.array(ListPred)
normListTrue = np.array(normListTrue)
normListPred = np.array(normListPred)

smapes_RW=[SMAPE(ListTrue[:,h], ListPred[:,h]) for h in range(4)]
mae_RW=[MAE(ListTrue[:,h], ListPred[:,h]) for h in range(4)]
normMae_RW=[MAE(normListTrue[:,h], normListPred[:,h]) for h in range(4)]

print(smapes_RW,np.mean(smapes_RW))
print(mae_RW,np.mean(mae_RW))
print(normMae_RW,np.mean(normMae_RW))

[15.139758412430925, 19.464677019817074, 24.74341415777439, 30.1103117407822] 22.364540332701146
[807.439, 1160.3903, 1500.1342, 1821.8292] 1322.4481
[0.13769865, 0.1978901, 0.2558292, 0.31069028] 0.22552705
